In [29]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Reshape
from keras.layers.embeddings import Embedding
from keras.layers import Merge
from keras.callbacks import ModelCheckpoint
import datetime,sys,os,time
import pandas as pd
import numpy as np
import sys,os

In [30]:
cate_cols = ['store_id_encoded', 'area_name', 'city', 'genre_name']
cate_feats = ['dow', 'hol_days']#, 'is_weekends', 'holiday_flg', 'month', 'is_up_corner']
# for mod in ['air', 'hpg']:
#     cate_feats.extend(['%s_%s' % (mod, c) for c in cate_cols])

def split_features(X):
    X_list = []
    for idx in range(len(cate_feats)):
        feat_vec = X[..., [idx]]
        X_list.append(feat_vec)        
    return X_list

class Model(object):

    def evaluate(self, X_val, y_val):
        assert(min(y_val) > 0)
        guessed_sales = self.guess(X_val)
        relative_err = np.absolute((y_val - guessed_sales) / y_val)
        result = np.sum(relative_err) / len(y_val)
        return result

class NN_with_EntityEmbedding(Model):
    def __init__(self, X_train, y_train, X_val, y_val, dim_dict):
        super().__init__()
        self.nb_epoch = 10
        self.input_dim_dict = dim_dict
        self.checkpointer = ModelCheckpoint(filepath="best_model_weights.hdf5", verbose=1, save_best_only=True)
        self.max_log_y = max(np.max(np.log(y_train)),  np.max(np.log(y_val)))
        self.__build_keras_model()
        self.fit(X_train, y_train, X_val, y_val)

    def preprocessing(self, X):
        X_list = split_features(X)
        return X_list

    def __build_keras_model(self):
        models = []

        model_dow = Sequential()
        model_dow.add(Embedding(self.input_dim_dict['dow'], 6, input_length=1))
        model_dow.add(Reshape(target_shape=(6,)))
        models.append(model_dow)

        model_holidays = Sequential()
        model_holidays.add(Embedding(self.input_dim_dict['hol_days'], 4, input_length=1))
        model_holidays.add(Reshape(target_shape=(4,)))
        models.append(model_holidays)

#         model_isweekends = Sequential()
#         model_isweekends.add(Dense(1, input_dim=1))
#         models.append(model_isweekends)
        
#         model_holidayflg = Sequential()
#         model_holidayflg.add(Dense(1, input_dim=1))
#         models.append(model_holidayflg)
           
#         model_month = Sequential()
#         model_month.add(Embedding(12, 6, input_length=1))
#         model_month.add(Reshape(target_shape=(6,)))
#         models.append(model_month)
        
#         model_isupcorner = Sequential()
#         model_isupcorner.add(Dense(1, input_dim=1))
#         models.append(model_isupcorner)
        
#         model_airstoreid = Sequential()
#         model_airstoreid.add(Embedding(self.input_dim_dict['air_store_id_encoded'], 10, input_length=1))
#         model_airstoreid.add(Reshape(target_shape=(10,)))
#         models.append(model_airstoreid)

#         model_airareaname = Sequential()
#         model_airareaname.add(Embedding(self.input_dim_dict['air_area_name'], 10, input_length=1))
#         model_airareaname.add(Reshape(target_shape=(10,)))
#         models.append(model_airareaname)

#         model_aircity = Sequential()
#         model_aircity.add(Embedding(self.input_dim_dict['air_city'], 6, input_length=1))
#         model_aircity.add(Reshape(target_shape=(6,)))
#         models.append(model_aircity)
        
#         model_airgenrename = Sequential()
#         model_airgenrename.add(Embedding(self.input_dim_dict['air_genre_name'], 6, input_length=1))
#         model_airgenrename.add(Reshape(target_shape=(6,)))
#         models.append(model_airgenrename)
        
#         model_hpgstoreid = Sequential()
#         model_hpgstoreid.add(Embedding(self.input_dim_dict['hpg_store_id_encoded'], 10, input_length=1))
#         model_hpgstoreid.add(Reshape(target_shape=(10,)))
#         models.append(model_hpgstoreid)

#         model_hpgareaname = Sequential()
#         model_hpgareaname.add(Embedding(self.input_dim_dict['hpg_area_name'], 10, input_length=1))
#         model_hpgareaname.add(Reshape(target_shape=(10,)))
#         models.append(model_hpgareaname)
        
#         model_hpgcity = Sequential()
#         model_hpgcity.add(Embedding(self.input_dim_dict['hpg_city'], 6, input_length=1))
#         model_hpgcity.add(Reshape(target_shape=(6,)))
#         models.append(model_hpgcity)
        
#         model_hpggenrename = Sequential()
#         model_hpggenrename.add(Embedding(self.input_dim_dict['hpg_genre_name'], 6, input_length=1))
#         model_hpggenrename.add(Reshape(target_shape=(6,)))
#         models.append(model_hpggenrename)
        
        self.model = Sequential()
        self.model.add(Merge(models, mode='concat'))
        self.model.add(Dense(100000, init='uniform'))
        self.model.add(Activation('relu'))
        self.model.add(Dense(5000, init='uniform'))
        self.model.add(Activation('relu'))
        self.model.add(Dense(1))
        self.model.add(Activation('sigmoid'))

        self.model.compile(loss='mean_absolute_error', optimizer='adam')

    def _val_for_fit(self, val):
        val = np.log(val) / self.max_log_y
        return val

    def _val_for_pred(self, val):
        return np.exp(val * self.max_log_y)

    def fit(self, X_train, y_train, X_val, y_val):
#         print(X_train[:20])
#         self.preprocessing(X_train)
#         self.preprocessing(X_val)
#         sys.exit(1)
        self.model.fit(self.preprocessing(X_train), self._val_for_fit(y_train), validation_data=(self.preprocessing(X_val), self._val_for_fit(y_val)), nb_epoch= self.nb_epoch, batch_size= 128)
        # self.model.load_weights('best_model_weights.hdf5')
        print("Result on validation data: ", self.evaluate(X_val, y_val))

    def guess(self, features):
        features = self.preprocessing(features)
        result = self.model.predict(features).flatten()
        return self._val_for_pred(result)

In [31]:
import json
import pandas as pd
import dill as pickle

class DataUtil2:
    """"""
    @classmethod
    def load(cls, file, format, date_cols= None):
        """"""
        data = ''
        if(format== 'csv'):
            data = pd.read_csv(file, parse_dates= date_cols)
        elif(format== 'json'):
            with open(file, 'r') as i_file:
                data = json.load(file)
            i_file.close()
        elif(format== 'pkl'):
            with open(file, 'rb') as i_file:
                data = pickle.load(i_file)
            i_file.close()
        elif(format == 'hdf'):
            data = pd.read_hdf(path_or_buf= file, key='undefined')

        return  data

    @classmethod
    def save(cls, data, file, format, precision= 8):
        """"""
        if(format == 'csv'):
            data.to_csv(file, float_format= '%%.%df' % precision, index= False)
        elif(format == 'json'):
            with open(file, 'w') as o_file:
                json.dump(data, o_file, ensure_ascii= True, indent= 4)
            o_file.close()
        elif(format == 'pkl'):
            with open(file, 'wb') as o_file:
                pickle.dump(data, o_file, -1)
            o_file.close()
        elif(format== 'hdf'):
            data.to_hdf(path_or_buf= file, key='undefined', mode='w', complib='blosc')

        return

In [32]:
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

from sklearn import preprocessing

DataBaseDir = '../../data'
InputDir = '%s/l0/kfold' % DataBaseDir
OutputDir = '%s/l1/kfold' % DataBaseDir
kfold = 5
strategy = 'NN_EF'
start_time = datetime.datetime.now()
#### load data
valid_dfs = []
holdout_dfs = []
test_dfs = []
for fold in range(kfold):
    FoldInputDir = '%s/%s' % (InputDir, fold)
    valid = pd.read_csv('%s/valid.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    holdout = pd.read_csv('%s/holdout.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    test = pd.read_csv('%s/test.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    valid['fold'] = fold
    valid_dfs.append(valid)
    holdout_dfs.append(holdout)
    test_dfs.append(test)
TrainData = pd.concat(valid_dfs, axis= 0, ignore_index= True) 

# print(TrainData[cate_feats].head(10))
##### model selection with CV
# score
cv_score = .0
holdout_score = .0
# predict
y_test_pred = 0
# parameters

start = time.time()
for fold in range(kfold):
#     for c in cate_feats:
#         lbl = preprocessing.LabelEncoder()
#         TrainData[c] = lbl.fit_transform(TrainData[c])
    FoldData = {
        'train': TrainData[TrainData['fold'] != fold],
        'valid': TrainData[TrainData['fold'] == fold],
        'holdout': holdout_dfs[fold],
        'test': test_dfs[fold]
    }
    #col = [c for c in FoldData['train'].columns if c not in drop_cols]
    # train
    input_dim_dict = {}
    for c in cate_feats:
        input_dim_dict[c] = len(TrainData[c].unique())
    print(cate_feats)
#     sys.exit(1)
    model = NN_with_EntityEmbedding(FoldData['train'][cate_feats].values, FoldData['train']['visitors'].values,FoldData['valid'][cate_feats].values, FoldData['valid']['visitors'].values, input_dim_dict)
    # for holdout
    FoldData['holdout'][strategy] = model.guess(FoldData['holdout'][cate_feats])
    rmsle_holdout = RMSLE(FoldData['holdout']['visitors'].values, FoldData['holdout'][strategy])
    holdout_score += rmsle_holdout
    # for test
    FoldData['test'][strategy] = model.guess(FoldData['test'][cate_feats])
    y_test_pred += FoldData['test'][strategy]

    print('fold %s: valid score %.6f, holdout score %.6f, valid length %s' % (fold, rmsle_valid, rmsle_holdout, len(FoldData['valid'])))  
    #### output
    FoldOutputDir = '%s/%s' % (OutputDir, fold)
    if(os.path.exists(FoldOutputDir) == False):
        os.makedirs(FoldOutputDir)
    for mod in FoldData.keys():
        if(mod == 'train'):
            continue
        OutCols = []
        if(mod == 'test'):
            OutCols.append('id')
        OutCols.extend(['air_store_id', 'visit_date', 'visitors', strategy])
        OutputFile = '%s/%s_%s.csv' % (FoldOutputDir, mod, strategy)
        OutFoldData = FoldData[mod][OutCols]
        OutFoldData.to_csv(OutputFile, index= False)
    end = time.time()
    print('saving for %sth fold data done, time elapsed %.2fs.' % (fold, (end - start)))
    
y_test_pred /= kfold  # Average test set predictions
cv_score /= kfold # Average valid set predictions
holdout_score /= kfold # Average holdout set predictions

# Create submission file
sub = pd.DataFrame()
sub['id'] = test_dfs[0]['id']
sub['visitors'] = np.expm1(y_test_pred)
OutputFileName = '%s_submit_%s' % (strategy, datetime.datetime.now().strftime("%Y-%m-%d"))
SubmitDir = '%s/l1/submit' % DataBaseDir
if(os.path.exists(SubmitDir) == False):
    os.makedirs(SubmitDir)
sub.to_csv('%s/%s.csv' % (SubmitDir, OutputFileName), float_format='%.6f', index=False)
print('zip %s/%s.zip %s/%s.csv' % (SubmitDir, OutputFileName, SubmitDir, OutputFileName))
os.system('zip %s/%s.zip %s/%s.csv' % (SubmitDir, OutputFileName, SubmitDir, OutputFileName))

finish_time = datetime.datetime.now()
elapsed = (finish_time - start_time).seconds
print('\n======================')
print("CV score %.4f, Holdout score %.4f, Elapsed time: %.2fs" % (cv_score, holdout_score, elapsed))
print('======================\n')

['dow', 'hol_days']


/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:107: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:108: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100000, kernel_initializer="uniform")`
/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:110: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(5000, kernel_initializer="uniform")`
/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 181482 samples, validate on 45371 samples
Epoch 1/10
   128/181482 [..............................] - ETA: 215012s - loss: 0.1472

InvalidArgumentError: indices[9,0] = 6 is not in [0, 5)
	 [[Node: embedding_16/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](embedding_16/embeddings/read, embedding_16/Cast)]]

Caused by op 'embedding_16/Gather', defined at:
  File "/Users/yuanpingzhou/miniconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/yuanpingzhou/miniconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/Users/yuanpingzhou/minicon